In [0]:
"""
WARNING
    > Remove unnecessary code from notebooks that would return results, such as display and count.
    > Do not run Structured Streaming workloads on interactive clusters; always schedule streams as jobs.
    > To help streaming jobs recover automatically, configure jobs with infinite retries.
    > Do not use auto-scaling for workloads with Structured Streaming.
"""

In [0]:
import pyspark.sql.functions as f
import pyspark.sql.utils
from pyspark.sql.types import *
from pyspark.sql.window import Window
import pandas as pd
import datetime

In [0]:
dbutils.widgets.text("startDate", "")
dbutils.widgets.text("endDate", "")

startDate = dbutils.widgets.get("startDate")
endDate = dbutils.widgets.get("endDate")

dates = [startDate, endDate]

print(dates)
print(startDate, endDate)

['2024-08-11', '2024-08-11']
2024-08-11 2024-08-11


In [0]:
# Reading from Delta Lake as a bulk
# Permissions should be given to roles, buckets etc 
file_path = "s3a://zalando-datalake-binary/event-types/data/shop.tracking.cart.change/"
raw_df = (
        spark.read
        .format("delta")
        .load(file_path)
        .withColumn("data_sink", f.lit("nakadi"))
        .filter(f.col("dt").between(*dates))
        .limit(5)
        )

raw_df.display()

client_id,user_agent,frontend_type,language,behavioural_gender,customer_age,customer_gender,customer_hash,timestamp,context_gender,flowId,document_location,document_referrer,sales_channel,parent_tile,root_entity_id,root_renderer,advertising_id,currency,customer_email,customer_number,parent_entity_id,position_index,vat,event_name,metadata,dt,entity_id,entity_id_path,price,size,shop_language,message_task_sub_uuid,device_consent_id,deeplink,allow_analytics,data_sink
b0d5cbff-0640-4317-a0a3-ffe363b90e56,Zalando/24.15.0 (Linux; Android 13; Xiaomi M2101K6G/TKQ1.221013.002 test-keys),mobile-app,es-ES,-,50,women,d07c02518afdd4a45c05801e7c3daddc,2024-08-11T18:50:34.447Z,women,c391d867-f362-4d79-ac,null,null,1e161d6e-0427-4cfc-a357-e2b501188a15,null,ern:product::PU141A103-C11,product_view,b56ebc5c-f6cf-4665-ad05-ba6231d5749a,EUR,e244d049eaa443c0e9dbb1cf8b0ad2ac8881eae4f76a574e2214c7dd582b4c23,31602181538,null,1,21.0,add-to-cart,"List(5d005bef-318f-461c-8803-9c6c13ea738f, shop.tracking.cart.change, 2024-08-11T18:51:06.140191273Z, null, null, njc-1723402266646-45aabcf21835c6bc, 9, 2.7.0, 2024-08-11T18:51:06.649Z, stups_tpi-composer)",2024-08-11,ern:product::PU141A103-C11,ern:product::PU141A103-C11,43.35,38,es-ES,null,f28043d9-5938-0f39-3fa1-c5b3e5913e3f,zalando://ROOT/Home,true,nakadi
a5296fb1-483f-4fc4-8c7d-38f2a631b954,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36",browser,es-ES,-,50,women,141ba7e9fcd8ca7ea8ab60eb8085a659,2024-08-11T18:50:33.779Z,women,ZHGKyBSlHBbVNYDN,https://www.zalando.es/bruuns-bazaar-raisella-larisa-top-vivacious-br321e07r-j11.html,https://www.zalando.es/ropa-de-mujer-camisetas-y-tops/_naranja.rosa/?price_from=30,1e161d6e-0427-4cfc-a357-e2b501188a15,product detail page,ern:product::BR321E07R-J11,product_view,null,EUR,b95942f0c81824ab5435d891010a20fea2bcb5b780f4005e288362b2ce0f4973,31600918823,ern:product::BR321E07R-J11,1.7,21.0,add-to-cart,"List(367e85af-bbcf-4aba-9247-79889d6840bb, shop.tracking.cart.change, 2024-08-11T18:51:06.565988251Z, null, null, njc-1723402267153-c204276e8aa44843, 9, 2.7.0, 2024-08-11T18:51:07.158Z, stups_tpi-composer)",2024-08-11,ern:product::BR321E07R-J11,ern:product::BR321E07R-J11/ern:product::BR321E07R-J11,57.81,40,es-ES,null,null,null,true,nakadi
6d51b568-9433-45df-8d28-e24b08c46ad1,zalando/24.12.0 (iPhone; CPU iOS 17.5.1; Scale/3.00),mobile-app,de-DE,-,35,men,8815ecd9ec4d4ffef955a27f0096b0b0,2024-08-11T18:50:32.763Z,men,4B4503D0-9029-4D85-8A,null,null,01924c48-49bb-40c2-9c32-ab582e6db6f4,null,ern:product::AD542D5IC-A11,product_view,null,EUR,983ff6dc63e13e28c872e23c7737feaf38cb0e51d92150363124ef84d72521df,3035046716,null,1,19.0,add-to-cart,"List(6553f4d6-c483-451a-8674-07a4d6b156d9, shop.tracking.cart.change, 2024-08-11T18:51:06.435465514Z, null, null, njc-1723402267354-b726a2b27ade3ffc, 9, 2.7.0, 2024-08-11T18:51:07.360Z, stups_tpi-composer)",2024-08-11,ern:product::AD542D5IC-A11,ern:product::AD542D5IC-A11,25.17,M,de-DE,null,765F7551-C5EA-4B42-ABC2-3B3F635F13EA,null,true,nakadi
5a935834-b4df-4d14-b40f-83c2730b8902,zalando/24.12.0 (iPhone; CPU iOS 17.5.1; Scale/3.00),mobile-app,sv-SE,-,52,women,6f1cbc01d1cd2b4e6abe3d58a009e486,2024-08-11T18:50:35.617Z,women,4A6EEFC1-888D-49C3-97,null,null,091dcbdd-7839-4f39-aa05-324eb4599df0,null,ern:product::MBQ51Q066-B11,product_view,null,SEK,f18c7f414bf88be4377f0960be9452b5017855b9b040e918f183fbafadeb1a79,31300646737,null,1,25.0,add-to-cart,"List(c46fa390-f510-44f4-993c-80d674522a91, shop.tracking.cart.change, 2024-08-11T18:51:07.573222044Z, null, null, njc-1723402267839-81bb592b94d54ca8, 9, 2.7.0, 2024-08-11T18:51:07.843Z, stups_tpi-composer)",2024-08-11,ern:product::MBQ51Q066-B11,ern:product::MBQ51Q066-B11,311.20,One Size,sv-SE,b7ce29f7-f633-4934-9ae4-976830ed77de,B4DFE0BE-5214-49D2-A838-35A7FF198E6B,https://www.zalando.se/levis-pack-large-unisex-ryggsaeck-black-le254o019-q11.html?wmc=CRM46_REC_SV.ONL_SBS_PMT_PEC_REC_a99c229d-b27d-4026-9fb3-cca740df322e_240811.&cd084=collection_reco

In [0]:
# Read from Delta Lake as Stream
file_path = "s3a://zalando-datalake-binary/event-types/data/shop.tracking.cart.change/"
checkpoint_read = "/tmp/events/checkpoint_read"

raw_stream_df = (
    spark.readStream
    .format("delta")
    .option("checkpointLocation", checkpoint_read)
    .load(file_path)
    .filter(f.col("dt").between(*dates))
    .limit(5)
)

raw_stream_df.display() # Calling display() on a streaming DataFrame starts a streaming job.

client_id,user_agent,frontend_type,language,behavioural_gender,customer_age,customer_gender,customer_hash,timestamp,context_gender,flowId,document_location,document_referrer,sales_channel,parent_tile,root_entity_id,root_renderer,advertising_id,currency,customer_email,customer_number,parent_entity_id,position_index,vat,event_name,metadata,dt,entity_id,entity_id_path,price,size,shop_language,message_task_sub_uuid,device_consent_id,deeplink,allow_analytics
afc0901f-ec4a-416b-a3bf-1932a7047f62,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36 Edg/127.0.0.0",browser,de,-,34,women,9a8aa489efadffd66101956728e95fd5,2024-08-10T23:59:31.174Z,women,oOwIbMihwca4AhOi,https://www.zalando.de/wardrobe/lists/liked/,https://www.zalando.de/wardrobe/,01924c48-49bb-40c2-9c32-ab582e6db6f4,wardrobe,ern:customer::self,customer_wardrobe-view,null,EUR,b0d12c8b4c44535fc981052957dc555a1476b68d0e6483a70ec07e4a76a99f23,46741129414459,ern:customer::self,1.13,19.0,add-to-cart,"List(53e3cebd-0cb6-4343-b70b-c7011e2a9f6f, shop.tracking.cart.change, 2024-08-11T00:00:02.817839257Z, null, null, njc-1723334402839-0dd9fe17aaa23d49, 15, 2.7.0, 2024-08-11T00:00:02.842Z, stups_tpi-composer)",2024-08-11,ern:product::L2E21B02G-Q11:wardrobe/56247fd5a1ea75e25fe298f8980723f4b2ce3d5989ed32d03a819952708e19b5,ern:customer::self/ern:product::L2E21B02G-Q11:wardrobe/56247fd5a1ea75e25fe298f8980723f4b2ce3d5989ed32d03a819952708e19b5,36.93,40/42,de-DE,null,null,null,true
c90e63cb-44d0-4dfc-b74c-cb74efc03c4d,zalando/24.14.0 (iPhone; CPU iOS 16.1.1; Scale/3.00),mobile-app,en-DE,-,null,women,4b311805c7053352ae44fc6c9bd85a48,2024-08-10T23:59:33.873Z,women,D5FF12EF-5E25-476B-97,null,null,01924c48-49bb-40c2-9c32-ab582e6db6f4,null,ern:product::AN681P0B4-F11,product_view,null,EUR,e247b6a904ae1c410c8ec8762b739f256d82bd64420196b1ce667d67437afde5,46490993504852,null,1,19.0,add-to-cart,"List(38ca74b3-ad2f-4eb4-a5f9-45f9212b1d0d, shop.tracking.cart.change, 2024-08-11T00:00:05.456838180Z, null, null, njc-1723334405643-25e264fc3deafdf2, 15, 2.7.0, 2024-08-11T00:00:05.646Z, stups_tpi-composer)",2024-08-11,ern:product::AN681P0B4-F11,ern:product::AN681P0B4-F11,21.00,S,en-GB,null,6A1F0051-2923-47B1-BD2F-DE2A57AEE104,null,true
afc0901f-ec4a-416b-a3bf-1932a7047f62,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36 Edg/127.0.0.0",browser,de,-,34,women,9a8aa489efadffd66101956728e95fd5,2024-08-10T23:59:34.304Z,women,oOwIbMihwca4AhOi,https://www.zalando.de/wardrobe/lists/liked/,https://www.zalando.de/wardrobe/,01924c48-49bb-40c2-9c32-ab582e6db6f4,wardrobe,ern:customer::self,customer_wardrobe-view,null,EUR,b0d12c8b4c44535fc981052957dc555a1476b68d0e6483a70ec07e4a76a99f23,46741129414459,ern:customer::self,1.14,19.0,add-to-cart,"List(07d63c37-c622-4dcf-8974-c02cfb90256d, shop.tracking.cart.change, 2024-08-11T00:00:05.805143999Z, null, null, njc-1723334405839-1ca760229264f719, 15, 2.7.0, 2024-08-11T00:00:05.842Z, stups_tpi-composer)",2024-08-11,ern:product::L2E21B02G-M11:wardrobe/c60b18fcbda360f7f0228a7446b3eaa00403ffecd3d61e46d2f2fd0717c21439,ern:customer::self/ern:product::L2E21B02G-M11:wardrobe/c60b18fcbda360f7f0228a7446b3eaa00403ffecd3d61e46d2f2fd0717c21439,33.57,40/42,de-DE,null,null,null,true
105c64bd-87f9-4e5f-b507-7bd474428f23,"Mozilla/5.0 (iPhone; CPU iPhone OS 17_5_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.5 Mobile/15E148 Safari/604.1",browser,de-DE,-,48,no preference,0db62b16728ee8ba4b9e7261b48890ec,2024-08-10T23:59:34.427Z,women,7Pmz8Y4mftJe-pQA,https://www.zalando.ch/sloggi-slip-black-sl281a01f-q11.html,https://www.zalando.ch/damenbekleidung-waesche-unterteile/,c2bd90da-0090-4751-8f16-35dea7002071,product detail page,ern:product::SL281A01F-Q11,product_view,null,CHF,3ebd544120e7bceb3700a6a6bf4befeedf1844c52b6f070a87839c4af29adba4,46576597213592,ern:product::SL281A01F-Q11,1.7,7.7,add-to-cart,"List(0e17f142-fdaa-4eb8-b835-5d54e18cd7d0, shop.tracking.cart

In [0]:
# Write Imcremantal batch to Delta Lake
# The "availableNow=True" setting for the trigger instructs Structured Streaming to process all previously unprocessed records from the source dataset and then shut down
# The checkpointLocation should be unique for each streaming writer. This provides the unique identity for your stream, tracking all records processed and state information associated with your streaming query.
target_path = "/tmp/ss-tutorial/"
checkpoint_write = "/tmp/events/checkpoint_write"
(
raw_stream_df.writeStream
    .trigger(availableNow=True) 
    .option("checkpointLocation", checkpoint_write)
    .option("path", target_path)
    .outputMode("append")
    .start()
)